In [7]:
%pip install --quiet \ pandas 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

In [ ]:
dados = [
    ("associação de Agricultores Familiares da Eco Comunidade do Assentamento 15 de Agosto (Afeca)", ["São Sebastião"], -15.91117477568832, -47.72142914924957)
    ()

]